In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import mne
from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse, write_inverse_operator)
from mne.stats import summarize_clusters_stc

import scipy
from scipy import stats
import pickle

from mayavi import mlab
from IPython.display import Image


In [ ]:
sensors = 'mag'
meg_MainFolder = "..\Data\MEG_Data\Data="
tmin, tmax = -0.1, 0.6

dataFolder = meg_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Data folder: ', dataFolder)

source_MainFolder = "..\SourceLocalization\SourceEstimates\Data="
sourceFolder = source_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Source folder: ', sourceFolder)

subjects_dir = '..\SourceLocalization\subjects\\'
print('Subjects directory: ', subjects_dir)

forwardModelsFolder = '..\SourceLocalization\ForwardModels\\'
print('Forward models folder: ', forwardModelsFolder)

spatialFiltersFolder = '..\SourceLocalization\SpatialFilters\\Data='
spatialFiltersFolder = spatialFiltersFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Spatial filters folder: ', spatialFiltersFolder)

statResultsFolder  = '..\SourceLocalization\Results\\Data='
statResultsFolder = statResultsFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Statisctics results folder: ', statResultsFolder)

classifiers_MainFolder = "..\Classifiers\Data="    
clsfFolder = classifiers_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Classifiers folder: ', clsfFolder)
    

# Pick the classifiers based on the their training window
if tmin == -0.8:
    peak_indices = [90, 120]
elif tmin == -0.1:
    peak_indices = [20, 50]
    
print('Peak indices: ', peak_indices)
    
# Task name for the classifiers
task_name = 'all_predLevel'
print('Task name: ', task_name)

In [ ]:
filename_ext = ''
if tmin == -0.8 and tmax == 0.6:
    filename_ext = '-elongated'
elif tmin == -0.8 and tmax == 1:
    filename_ext = '-elongated_2'
    
print('filename ext: ', filename_ext)

In [ ]:
# All subjects
s_id_list_all = ['13', '16', '17', '18', '21', '23', '26', '28', '29', '30', 
                 '31', '32', '33', '34', '35', '36', '38', '39', '40', '41', '42']


participant_names = ['Vinitski_D', 'Scherbakova_E', 'Ganbarov_E', 'Kasa_I' , 'Matuilko_I', 'Riaz_A', 'Solev_I',
                     'Kolzhanov_N', 'Kabanova_V', 'Mochenova_M', 'Novikov_A', 'Adamovich_T', 'Gornostaeva_E',
                     'Kazakov_I', 'Glebko', 'korobova', 'Ziberova_A', 'Prizhimova_E', 'Voronova_A', 'Saidov_F',
                     'Bolgina_T']


print('Number of subjects: ', len(participant_names))

if os.path.exists(subjects_dir + '\\' + participant_names[0] + '\surf\\'):  
    print('exists!')





In [ ]:
#participant_names[9]

In [ ]:
def getEpochData(s_id):
    
   
    if int(s_id) < 23:
        fname = dataFolder+'S'+s_id+'\\'+s_id+'_2_tsss_mc_trans_'+sensors+'_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'
    else: 
        fname = dataFolder+'S'+s_id+'\\block_2_tsss_mc_trans_'+sensors+'_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'


    epochs = mne.read_epochs(fname, verbose='error')
    print(fname + ' loaded!')
    return epochs


In [ ]:
def getClassifierWeghts(filename):

    print('Classifier is loaded from ', filename)
    loaded_model = []
    with open(filename, "rb") as f:
        while True:
            try:
                loaded_model.append(pickle.load(f))
            except EOFError:
                break
        
    return loaded_model

In [ ]:
def splitData(epochs, events=None):
    #print(epochs.event_id)
    if events == None:
        print('No events are given as parameter!')
    
    else:
        print('Requested events: ', events)
        return_list = epochs[events]
    
    print('Events in return list: ', return_list.event_id)
    
    return return_list



# Inverse modeling with BeamFormer on evoked data

## Beamformer

In [ ]:
# Compute spatial filters for beamformer

def computeSpatialFilter(s_name, s_id, evoked, noise_cov, data_cov, condName):
    print('Beamformer')
    print('Computing spatial filter..')
    
    # Load forward model
    fname_fwd = forwardModelsFolder + '\\fwd_sol_ico5_' + s_name + '.fif'
    fwd = mne.read_forward_solution(fname_fwd, verbose=False)
    
    # Compute spatial filters with evoked data, forward model and covariance matrices
    filters = mne.beamformer.make_lcmv(evoked.info, fwd, data_cov=data_cov, reg=0.05, noise_cov=noise_cov, 
                                       pick_ori='max-power', weight_norm='unit-noise-gain', rank='full', reduce_rank=True)
   
    # Save filters
    filters.save(spatialFiltersFolder + s_id + '_filters-lcmv_' + s_name + '.h5', overwrite=True)
    return filters
    

### Prepare data

#### Morph subject's source estimate to template subject

In [ ]:
def morphToCommonSpace(stc, s_name, src_ave, smoothAmount=None):
    print('Computing source morph..')
    # Read the source space we are morphing to
    fsave_vertices = [s['vertno'] for s in src_ave]

    morph = mne.compute_source_morph(src=stc, subject_from=s_name, subject_to='fsaverage', 
                                     spacing=fsave_vertices, subjects_dir=subjects_dir, verbose=False,
                                     smooth=smoothAmount)


    tstep = stc.tstep
    
    print('Morphing data to fsaverage..')
    stc_fsave = morph.apply(stc)
    n_vertices_fsave = morph.morph_mat.shape[0]
    
    
    return stc_fsave, n_vertices_fsave, tstep

    

In [ ]:
def computeEvoked_averagedTimeRange(epochs, timeRange):
    
    epochs = epochs.crop(tmin=timeRange[0], tmax=timeRange[1])
    epochs_data = epochs.get_data()
    print('epochs data shape: ', epochs_data.shape)
    
    epochs_data_avgTimeRange = np.mean(epochs_data, axis=2)
    print('epochs_data_avgTimeRange shape: ', epochs_data_avgTimeRange.shape)
    
    evoked_data = np.mean(epochs_data_avgTimeRange, axis=0)
    evoked_data = np.reshape(evoked_data, (len(evoked_data), 1))
    print('evoked_data shape: ', evoked_data.shape)
    
    evoked = mne.EvokedArray(evoked_data, epochs.info)
    
    return evoked
    

In [ ]:
# generate the inverse solution for group average
def prepareInverseSolution_group(data, tstep, tmin_tmp=0):
    
    src_ave = mne.read_source_spaces(subjects_dir+'fsaverage\\bem\\fsaverage-ico-5-src.fif')

    fsave_vertices = [s['vertno'] for s in src_ave]        
            
    stc_return = mne.SourceEstimate(data, fsave_vertices, tmin_tmp, tstep, subject='fsaverage')
    
    
    return stc_return
    

### Visualize results

The STC (Source Time Courses) are defined on a source space formed by 7498 candidate locations and for a duration spanning 106 time instants.

Warning: Slide Type
!!PQt5 is necessary and also run jupyter nbextension enable mayavi --py on command line before running the jupyter notebooks and also latest (6.1.1) version of module called 'traits'.

In [ ]:
mlab.init_notebook('png')
mne.viz.set_3d_backend('mayavi') #pyvista
def showResult(s_id, sourceFolder, stc, condName, minimum, maximum, mid, tmin_tmp=0, 
               transparent=True, cmap='mne', alpha=1.0, center=1):
    
    initial_time = tmin_tmp
    hemi_list = ['rh', 'lh']
    for hemi in hemi_list:
        print('Hemi: ', hemi)
        
        kwargs = dict(initial_time=initial_time, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=False, size=(600, 600), spacing='all', background='w',
                      cortex=(211/256,211/256,211/256), colorbar=True) #seismic coolwarm
        
        #plt.clim(minimum, maximum) 
        if cmap  == 'mne' or  cmap == 'Reds':
             brain = stc.plot(**kwargs, colormap=cmap, clim=dict(kind='value', lims=[minimum, mid, maximum]))  
        else:
            brain = stc.plot(**kwargs, colormap=cmap, clim=dict(kind='value', pos_lims=[minimum, mid, maximum]))        
        #brain.scale_data_colormap(fmin=minimum, fmid=mid, fmax=maximum, 
        #                             transparent=transparent, alpha=alpha, center=center)
        
   
        brain.show_view('lateral');

        brain.save_image(sourceFolder + s_id + '_' + hemi + '_' + condName + '.png')

        Image(filename = sourceFolder +  s_id + '_' + hemi + '_' + condName + '.png', width=600)


### Apply Beamformer

In [ ]:
from mne.decoding import get_coef
def computeActivationMaps(model_list, epochs, tmin):

    meg_data = epochs.get_data()
    epochs.average().plot()
    print("Meg data shape: ", meg_data.shape)
    
    # get classifier weights
    if len(model_list) > 0:
        model = model_list[0] # if model is loaded, it is stored in a list. Therefore we need to get model from index 0
        
        # Get classifier weights
        coef = get_coef(model, 'coef_', inverse_transform=True)
        
        # Compute mean and std of weights
        coef_mean = np.mean(coef)
        coef_std = np.std(coef)
        
        # Standardize the weights
        coef = (coef-coef_mean)/coef_std
        print('shape of coef: ', coef.shape)
    
    
    # Multiplying classifier weights with covariance of data to compute activation maps
    activations_mat = np.zeros((meg_data.shape[0], meg_data.shape[1], meg_data.shape[2]))
    # ntrials, nchannels, ntimes
    
    for t in range(meg_data.shape[2]):
        epochs_tmp = epochs.copy()
        epochs_tmp.crop(tmin=epochs.times[t], tmax=epochs.times[t])
        cov_tmp = mne.compute_covariance(epochs_tmp, verbose=False)

        activations = np.dot(coef, cov_tmp.data)
        if t == 0:
            print('Shape of activations: ', activations.shape)
        
        for i in range( meg_data.shape[0]):
            activations_mat[i, :,t] = activations.reshape(meg_data.shape[1])
            
        del cov_tmp 
    
    # Simulate epoch object with activation maps
    epoched_sim = mne.EpochsArray(activations_mat, epochs.info, tmin=tmin)

    return epoched_sim


### Statistics to compare conditions

In [ ]:
def computeStatistic(x, y):
 
    print('comparing 2 groups')
    '''
    for i in range(x.shape[0]):
        #print('vertex ind: ', i)
        stats, pval = scipy.stats.ttest_rel(x[i,:], y[i,:])
        
        stats_array[i] = stats
        pval_array[i] = pval
    '''
    stats_array, pval_array = stats.ttest_rel(x, y, axis=1)

    return stats_array, pval_array
    
    
    

In [ ]:

def applyBeamformer(conditions, s_id_list_all, n_subjects, participant_names,
                    tminData, tmaxData, tminNoise, tmaxNoise, tminEpoch, smoothAmount, task_name):
    
    stc_fsave_all, n_times = None,  None


    for s in range(n_subjects): 
        s_id = s_id_list_all[s]
        s_name = participant_names[s]
        print(' ------------- ' + s_name + ' ------------- ')
        epochs = getEpochData(s_id)
        print(epochs.event_id)
        print('epochs shape: ', epochs.get_data().shape)
        
        # check if all conditions exist in the epoch (e.g. omissions_living_nores might not exist!)
        conditions = [c for c in conditions if c in epochs.event_id]
        print('Final conditions: ', conditions)
        splits = epochs[conditions]
        print('Events in splits: ', splits.event_id)
        
                
        # Load classifier weights to compute activation maps
        clsf_model_filename = clsfFolder+'S'+s_id+'\\'+s_id+'_clsf_'+task_name+'_'+str(peak_indices[0])+'_'+str(peak_indices[1])+'_sm.sav'
        clsf_model = getClassifierWeghts(clsf_model_filename)

        # compute activation maps and simulate epoch object for source localization
        print('Compute activations')
        epoch_sim = computeActivationMaps(clsf_model, splits, tmin=tminEpoch)

        print('Compute noise covariance')
        noise_cov = mne.compute_covariance(epoch_sim, tmin=tminNoise, tmax=tmaxNoise, 
                                           method=['shrunk', 'empirical'], verbose=False) 
        print('Compute data covariance')
        data_cov = mne.compute_covariance(epoch_sim, tmin=tminData, tmax=tmaxData, 
                                          method=['shrunk', 'empirical'], verbose=False)

        # compute average of epochs
        evoked = epoch_sim.average().crop(tmin=tminData, tmax=tmaxData)
        print('Shape of evoked data: ', evoked._data.shape)

        # computer spatial filters by LCMV
        print('Compute filter: ')
        filters = computeSpatialFilter(s_name, s_id, evoked, noise_cov, data_cov, conditions)
        
        print('Apply beamformer: ')
        stc = mne.beamformer.apply_lcmv(evoked=evoked, filters=filters, max_ori_out='signed')
        n_vertices_sample, n_times = stc.data.shape
       

        if s_id != '16' and s_id != '31' and s_id != '40':
            stc_fsave, n_vertices_fsave, tstep = morphToCommonSpace(stc, s_name, src_ave,
                                                                    smoothAmount=smoothAmount)
        else:
            stc_fsave = stc
            
        
        if s==0:
            print('n_times: ', n_times)
            stc_fsave_all = np.zeros((n_vertices_fsave, n_times, n_subjects),)

        
        stc_fsave_all[:,:,s] = np.abs(stc_fsave.data.reshape(n_vertices_fsave, n_times))




    return  stc_fsave_all, n_times, tstep
                    
            

#### Read fsaverage

In [ ]:
# Read fsaverage
src_ave = mne.read_source_spaces(subjects_dir + 'fsaverage\\bem\\fsaverage-ico-5-src.fif')
#fsave_vertices = [s['vertno'] for s in src_ave]


#### Choose the conditions for contrasting

In [ ]:
conditions_8 = ['living_omission_8_nores', 'living_omission_8_corr', 'living_omission_8_incorr',
                 'object_omission_8_nores', 'object_omission_8_corr', 'object_omission_8_incorr']

conditions_10 = ['living_omission_10_nores', 'living_omission_10_corr', 'living_omission_10_incorr',
                 'object_omission_10_nores', 'object_omission_10_corr', 'object_omission_10_incorr']
condition_names =  ["omi"]

n_subjects = len(s_id_list_all)
print('number of subjects: ', n_subjects)

print('conditions 80%: \n', conditions_8)
print('conditions 100%: \n', conditions_10)


### Define time limits for inverse solutins

In [ ]:
tminData, tmaxData = 0.1, 0.4  #0.1, 0.4 #-0.04, 0
print('tmin for data: ', tminData)
print('tmax for data: ', tmaxData)
tminNoiseCov, tmaxNoiseCov = -0.1, -0.05
print('tmin for noise covariance: ', tminNoiseCov)
print('tmax for noise covariance: ', tmaxNoiseCov)
tminEpoch = -0.1
print('tmin for generated epochs: ', tminEpoch)
smoothAmount = 70
print('Smoothing amount: ', smoothAmount)
inv_sol_method = 'beamformer'
print('Inverse solution method: ', inv_sol_method)
tstep=0.01

### Use beamformer for computing source estimates

80%

In [ ]:
print('======= Applying Beamformer ========')

stc_fsave_omi_8, n_times, tstep = applyBeamformer(conditions_8, s_id_list_all, 
                                                     n_subjects, participant_names, tminData, tmaxData, 
                                                     tminNoiseCov, tmaxNoiseCov, tminEpoch, smoothAmount, task_name)


stc_omi_8_filename = 'stc_fsave_omi_8_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'

print('stc omi is saved in ', stc_omi_8_filename)
np.save(statResultsFolder + stc_omi_8_filename, stc_fsave_omi_8)


100%

In [ ]:
print('======= Applying Beamformer ========')

stc_fsave_omi_10, n_times, tstep = applyBeamformer(conditions_10, s_id_list_all, 
                                                     n_subjects, participant_names, tminData, tmaxData, 
                                                     tminNoiseCov, tmaxNoiseCov, tminEpoch, smoothAmount, task_name)


stc_omi_10_filename = 'stc_fsave_omi_10_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'

print('stc omi is saved in ', stc_omi_10_filename)
np.save(statResultsFolder + stc_omi_10_filename, stc_fsave_omi_10)


In [ ]:

inv_sol_method

### If you have the source estimates values saved already, load them

cls time range

In [ ]:
tminData_cls_tmp, tmaxData_cls_tmp = 0.1, 0.4

# 80%
stc_omi_8_filename = 'stc_fsave_omi_8_onActivationMaps_'+inv_sol_method+'_'+str(tminData_cls_tmp)+'_'+str(tmaxData_cls_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(statResultsFolder+stc_omi_8_filename)
stc_fsave_omi_8_clfRange = np.load(statResultsFolder+stc_omi_8_filename)


print('shape of omi 80% source estimates: ', stc_fsave_omi_8_clfRange.shape)
print(np.where(stc_fsave_omi_8_clfRange == 0))


# 100%
stc_omi_10_filename = 'stc_fsave_omi_10_onActivationMaps_'+inv_sol_method+'_'+str(tminData_cls_tmp)+'_'+str(tmaxData_cls_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(statResultsFolder+stc_omi_10_filename)
stc_fsave_omi_10_clfRange = np.load(statResultsFolder+stc_omi_10_filename)


print('shape of omi 100% source estimates: ', stc_fsave_omi_10_clfRange.shape)
print(np.where(stc_fsave_omi_10_clfRange == 0))

Baseline

In [ ]:
tminData_baseline_tmp, tmaxData_baseline_tmp = -0.04, 0

# 80%
stc_omi_8_filename_baseline = 'stc_fsave_omi_8_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(stc_omi_8_filename_baseline)
stc_fsave_omi_8_baseline = np.load(statResultsFolder+stc_omi_8_filename_baseline)
print('shape of omi 80% baseline source estimates: ', stc_fsave_omi_8_baseline.shape)
print(np.where(stc_fsave_omi_8_baseline == 0))

# 100%
stc_omi_10_filename_baseline = 'stc_fsave_omi_10_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
print(stc_omi_10_filename_baseline)
stc_fsave_omi_10_baseline = np.load(statResultsFolder+stc_omi_10_filename_baseline)
print('shape of omi 100% baseline source estimates: ', stc_fsave_omi_10_baseline.shape)
print(np.where(stc_fsave_omi_10_baseline == 0))

### Check data before doing anything!!

In [ ]:
#stc_fsave_omi_8_baseline

In [ ]:
#stc_fsave_omi_10_baseline

In [ ]:
#stc_fsave_omi_8_clfRange

In [ ]:
#stc_fsave_omi_10_clfRange

In [ ]:
stc_fsave_omi_8_clfRange.shape

# Compute group-level relative change

### 1) Take mean 

In [ ]:
# 80%
# take mean over participants (dimension 2)
stc_fsave_omi_8_clfRange_avg = np.mean(stc_fsave_omi_8_clfRange, axis=2)
print('Shape stc_fsave_omi_8_clfRange_avg after avg over subjects: ', stc_fsave_omi_8_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_8_clfRange_avg= np.mean(stc_fsave_omi_8_clfRange_avg, axis=1)
print('Shape stc_fsave_omi_8_clfRange_avg after avg across time: ', stc_fsave_omi_8_clfRange_avg.shape)
print('stc_fsave_omi_8_clfRange_avg shape: ', stc_fsave_omi_8_clfRange_avg.shape)

print('stc_fsave_omi_8_clfRange_avg: ', stc_fsave_omi_8_clfRange_avg)

# take mean over participants (dimension 2)
stc_fsave_omi_8_baseline_avg = np.mean(stc_fsave_omi_8_baseline, axis=2)
print('Shape stc_fsave_omi_8_baseline_avg after avg over subjects: ', stc_fsave_omi_8_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_8_baseline_avg = np.mean(stc_fsave_omi_8_baseline_avg, axis=1)
print('Shape stc_fsave_omi_8_baseline_avg after avg across time: ', stc_fsave_omi_8_baseline_avg.shape)
print('stc_fsave_omi_8_baseline_avg shape: ', stc_fsave_omi_8_baseline_avg.shape)

print('stc_fsave_omi_8_baseline_avg: ', stc_fsave_omi_8_baseline_avg)

In [ ]:
# 100%
# take mean over participants (dimension 2)
stc_fsave_omi_10_clfRange_avg = np.mean(stc_fsave_omi_10_clfRange, axis=2)
print('Shape stc_fsave_omi_10_clfRange_avg after avg over subjects: ', stc_fsave_omi_10_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_10_clfRange_avg= np.mean(stc_fsave_omi_10_clfRange_avg, axis=1)
print('Shape stc_fsave_omi_10_clfRange_avg after avg across time: ', stc_fsave_omi_10_clfRange_avg.shape)
print('stc_fsave_omi_10_clfRange_avg shape: ', stc_fsave_omi_10_clfRange_avg.shape)

print('stc_fsave_omi_10_clfRange_avg: ', stc_fsave_omi_10_clfRange_avg)

# take mean over participants (dimension 2)
stc_fsave_omi_10_baseline_avg = np.mean(stc_fsave_omi_10_baseline, axis=2)
print('Shape stc_fsave_omi_10_baseline_avg after avg over subjects: ', stc_fsave_omi_10_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_omi_10_baseline_avg = np.mean(stc_fsave_omi_10_baseline_avg, axis=1)
print('Shape stc_fsave_omi_10_baseline_avg after avg across time: ', stc_fsave_omi_10_baseline_avg.shape)
print('stc_fsave_omi_10_baseline_avg shape: ', stc_fsave_omi_10_baseline_avg.shape)

print('stc_fsave_omi_10_baseline_avg: ', stc_fsave_omi_10_baseline_avg)

### 2) Generate stc group

In [ ]:
#80%
stc_cls_8 = prepareInverseSolution_group(stc_fsave_omi_8_clfRange_avg, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_baseline_8 = prepareInverseSolution_group(stc_fsave_omi_8_baseline_avg, tstep=tstep,
                                            tmin_tmp=tminData_baseline_tmp)

#100%
stc_cls_10 = prepareInverseSolution_group(stc_fsave_omi_10_clfRange_avg, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_baseline_10 = prepareInverseSolution_group(stc_fsave_omi_10_baseline_avg, tstep=tstep,
                                            tmin_tmp=tminData_baseline_tmp)

### Plot

In [ ]:
showResult('fsaverage', sourceFolder, stc_cls_8,
           'omi_8_clf_'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=1300000, maximum=3000000)

showResult('fsaverage', sourceFolder, stc_baseline_8, 
           'omi_8_baseline_'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=99000, maximum=440000)

showResult('fsaverage', sourceFolder, stc_cls_10,
           'omi_10_clf_'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=1300000, maximum=3000000)

showResult('fsaverage', sourceFolder, stc_baseline_10, 
           'omi_10_baseline_'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=99000, maximum=440000)

### 3) Average relative change

In [ ]:
#80% avg

stc_fsave_omi_8_diff_avg = 100*(stc_fsave_omi_8_clfRange_avg-stc_fsave_omi_8_baseline_avg)/stc_fsave_omi_8_baseline_avg


# 100% avg

stc_fsave_omi_10_diff_avg = 100*(stc_fsave_omi_10_clfRange_avg-stc_fsave_omi_10_baseline_avg)/stc_fsave_omi_10_baseline_avg

print('Minimum of 80%: ', min(stc_fsave_omi_8_diff_avg))
print('Maximum of 80%: ', max(stc_fsave_omi_8_diff_avg))

print('Minimum of 100%: ', min(stc_fsave_omi_10_diff_avg))
print('Maximum of 100%: ', max(stc_fsave_omi_10_diff_avg))
    
    

In [ ]:
#thresh=145
thresh_8 = 770 #160
thresh_10 = 770
print(np.where(stc_fsave_omi_8_diff_avg >thresh_8)[0].shape)
print(np.where(stc_fsave_omi_10_diff_avg >thresh_10)[0].shape)


In [ ]:
# 80%
for i in range(stc_fsave_omi_8_diff_avg.shape[0]):
    if stc_fsave_omi_8_diff_avg[i] < thresh_8:
        stc_fsave_omi_8_diff_avg[i] = 0
        
# 100%
for i in range(stc_fsave_omi_10_diff_avg.shape[0]):
    if stc_fsave_omi_10_diff_avg[i] < thresh_10:
        stc_fsave_omi_10_diff_avg[i] = 0

In [ ]:
stc_diff_8 = prepareInverseSolution_group(stc_fsave_omi_8_diff_avg, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_diff_10 = prepareInverseSolution_group(stc_fsave_omi_10_diff_avg, tstep=tstep, tmin_tmp=tminData_cls_tmp)


In [ ]:
 '2omi_8_change_'+inv_sol_method+'_sm='+str(smoothAmount) +'_pthresh=' + str(thresh_8)

In [ ]:
#80%
print('Minimum of 80%: ', min(stc_diff_8.data))
print('Maximum of 80%: ', max(stc_diff_8.data))
showResult('fsaverage', sourceFolder, stc_diff_8, 
           '2omi_8_change_'+inv_sol_method+'_sm='+str(smoothAmount) +'_pthresh=' + str(thresh_8), 
           minimum=600, maximum=1100, transparent=True, cmap='Oranges')

#100%
print('Minimum of 100%: ', min(stc_diff_10.data))
print('Maximum of 100%: ', max(stc_diff_10.data))
showResult('fsaverage', sourceFolder, stc_diff_10, 
           '2omi_10_change_'+inv_sol_method+'_sm='+str(smoothAmount) +'_pthresh=' + str(thresh_10), 
           minimum=600, maximum=1100, transparent=True, cmap='Oranges')

### find the mni coordinates of the significant blob in the plot comparing 80  and 100

1) Get labels of parcellation

In [ ]:
#sd = '/Users/athina/Documents/Projects/Pinar_MEG/'

src_ave = mne.read_source_spaces(subjects_dir+'fsaverage\\bem\\fsaverage-ico-5-src.fif')

fsave_vertices = [s['vertno'] for s in src_ave]

label = mne.read_labels_from_annot('fsaverage', parc='aparc',hemi = 'both',
                                   subjects_dir=subjects_dir)
print('All labels: ')
label

In [ ]:
# Initialize a list of labels:

labels_names = stc_f_omi.data.shape[0]*[None]
labels_inSourceEst = stc_f_omi.data.shape[0]*[None]

for l_i in range(len(label)):

#    l_i = 0 # label index we want

    # the name for current labels:
    for ver in label[l_i].restrict(src_ave).vertices:
        labels_names[ver] = label[l_i].restrict(src_ave).name
        labels_inSourceEst[ver] = label[l_i].restrict(src_ave)


In [ ]:
desired_thres = 0 # find the peak for t-values supporting 100% > 80%

sign_i = np.where(stc_f_omi.data > desired_thres)[0]
labels_names_sign = []
for s in sign_i:
    print(labels_names[s])
    labels_names_sign.append(labels_names[s])
    
print('Number of significant labels', len(sign_i))

In [ ]:
hemi = 'rh'
hemi_ind = 1 #rh

    
# find the peak on the rh
peak_vertex, peak_time = stc_f_omi.get_peak(hemi=hemi)
print('peak_vertex: ', peak_vertex)

# get the vertex at the peak
peak_vertex_surf = stc_f_omi.rh_vertno[peak_vertex]

# plot the source estimate with the peak painted blue
kwargs = dict(initial_time=0.1, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all', background='w', 
              cortex=(211/256,211/256,211/256)) #, clim=dict(kind='value', pos_lims=(10, 100, 400))
        
brain = stc_f_omi.plot(**kwargs, colormap='Oranges')
brain.scale_data_colormap(fmin=v_mi, fmid=(v_mi+v_ma)/2, fmax=v_ma, transparent=True)

# add the vertex at the peak to the plot
#brain.add_foci(peak_vertex_surf, coords_as_verts=True, hemi=hemi, color='blue')

view = 'lateral' #medial lateral
brain.show_view(view);
brain.save_image(sourceFolder +  'onActivationMaps_omi_8vs10_' + hemi + '_with_maxpoint_'+view+'.png')

# convert vertex to MNI coordinates
coordinate = mne.vertex_to_mni(peak_vertex_surf, hemis=hemi_ind, subject='fsaverage', subjects_dir=subjects_dir)

print('coordinates: ', coordinate)




# Compute relative change for each participant at 80 and 100 level

### 1) Compute relative change

In [ ]:
#80%
# collapse time range by averaging over time points -> output will have the size of n_vertices x nsubjects
stc_fsave_omi_8_clfRange_avgOverTime = np.mean(stc_fsave_omi_8_clfRange, axis=1)
stc_fsave_omi_8_baseline_avgOverTime = np.mean(stc_fsave_omi_8_baseline, axis=1)

stc_fsave_omi_8_diff = np.zeros(stc_fsave_omi_8_clfRange_avgOverTime.shape)
print('shape of stc_fsave_omi_8_diff: ', stc_fsave_omi_8_diff.shape)
for j in range(stc_fsave_omi_8_diff.shape[1]):
    stc_fsave_omi_8_diff[:,j] = 100*(stc_fsave_omi_8_clfRange_avgOverTime[:,j]-stc_fsave_omi_8_baseline_avgOverTime[:,j])/stc_fsave_omi_8_baseline_avgOverTime[:,j]

print('stc_fsave_omi_8_diff: ', stc_fsave_omi_8_diff.shape)

# 100%
# collapse time range by averaging over time points -> output will have the size of n_vertices x nsubjects
stc_fsave_omi_10_clfRange_avgOverTime = np.mean(stc_fsave_omi_10_clfRange, axis=1)
stc_fsave_omi_10_baseline_avgOverTime = np.mean(stc_fsave_omi_10_baseline, axis=1)

stc_fsave_omi_10_diff = np.zeros(stc_fsave_omi_10_clfRange_avgOverTime.shape)

for j in range(stc_fsave_omi_10_diff.shape[1]):
    stc_fsave_omi_10_diff[:,j] = 100*(stc_fsave_omi_10_clfRange_avgOverTime[:,j]-stc_fsave_omi_10_baseline_avgOverTime[:,j])/stc_fsave_omi_10_baseline_avgOverTime[:,j]

print('stc_fsave_omi_10_diff: ', stc_fsave_omi_10_diff)  
    
    

In [ ]:
print(stc_fsave_omi_8_diff[0,0])
print(stc_fsave_omi_10_diff[0,0])

In [ ]:
for i in range(stc_fsave_omi_10_diff.shape[1]):
    print('i: ', i)
    print('N=', len(np.where(stc_fsave_omi_8_diff[:,i]-stc_fsave_omi_10_diff[:,i]>0)[0]))

### 2) Stats to contrast 80% predictability and 100% predictability

In [ ]:
max(fvalues_o_new[fvalues_o_new != 0])

In [ ]:
pthresh = 0.05
print('computing the stats..')

#omissions
fvalues_o, pvalues_o = computeStatistic(stc_fsave_omi_10_diff, stc_fsave_omi_8_diff)

# save values for omissions
fvalue_filename_o = statResultsFolder+'fvalues_all_omi_activations_8vs10_'+inv_sol_method+'_sm='+str(smoothAmount)+'.npy'
print(fvalue_filename_o)
pvalue_filename_o = statResultsFolder+'pvalues_all_omi_activations_8vs10_'+inv_sol_method+'_sm='+str(smoothAmount)+'.npy'
print('saving stats...')
np.save(fvalue_filename_o, fvalues_o)
np.save(pvalue_filename_o, pvalues_o)

print('Significant p vals omi: ', np.where(pvalues_o <= pthresh))
fvalues_o_new = np.copy(fvalues_o)
pvalues_o_new = np.copy(pvalues_o)



for j in range(len(fvalues_o)):
    #omissions
    if pvalues_o[j] > pthresh:
        pvalues_o_new[j] = 1
        fvalues_o_new[j] = 0

#omissions
rejets_o, pvalues_o_corrected = mne.stats.fdr_correction(pvalues_o)
print('omissions-corrrected:')
print(np.where(pvalues_o_corrected<= pthresh))


print('generate stc for stats..')
# omissions
stc_f_omi = prepareInverseSolution_group(fvalues_o_new, tstep=tstep)
#stc_f_omi = prepareInverseSolution_group(1-pvalues_o_new, tstep=tstep)

#omissions
#lateral
v_mi, v_ma = 0, 2.09  #-1.5, 1.5
showResult('fsaverage', sourceFolder, stc_f_omi, 
           'pval_predLevel_omi_activations_8vs10_pthresholded_'+inv_sol_method+'_sm='+str(smoothAmount)+'_pthresh='+str(pthresh),
           minimum=v_mi, maximum=v_ma, mid=(v_mi+v_ma)/2, transparent=False, cmap='coolwarm') #twilight_shifted


#del fvalues_o, pvalues_o, pvalues_o_corrected, fvalues_o_new, stc_f_omi

In [ ]:
print('fvalues_o_new min: ', min(abs(fvalues_o_new[fvalues_o_new != 0])))
print('fvalues_o_new max: ', max(abs(fvalues_o_new[fvalues_o_new != 0])))

In [ ]:
a= (1-pvalues_o_new)
a[np.where(a != 0)[0]]

Get  mni coordinates 

In [ ]:
hemi = 'rh'
hemi_ind = 1 #rh

    
# find the peak on the rh
peak_vertex, peak_time = stc_f_omi.get_peak(hemi=hemi)
print('peak_vertex: ', peak_vertex)

# get the vertex at the peak
peak_vertex_surf = stc_f_omi.rh_vertno[peak_vertex]


# convert vertex to MNI coordinates
coordinate = mne.vertex_to_mni(peak_vertex_surf, hemis=hemi_ind, subject='fsaverage', subjects_dir=subjects_dir)

print('coordinates: ', coordinate)




In [ ]:
# mni coordinates of frontal part

In [ ]:
#sd = '/Users/athina/Documents/Projects/Pinar_MEG/'

src_ave = mne.read_source_spaces(subjects_dir+'fsaverage\\bem\\fsaverage-ico-5-src.fif')

fsave_vertices = [s['vertno'] for s in src_ave]

label = mne.read_labels_from_annot('fsaverage', parc='aparc',hemi = 'both',
                                   subjects_dir=subjects_dir)
print('All labels: ')
label

In [ ]:
hemi = 'rh'
hemi_ind = 1

# find the labels of desired parcellations (e.g. frontal ones) 
labels_rh_frontal = [l for l in label if 'frontal' in l.name and hemi in l.name][2]
print(labels_rh_frontal)

In [ ]:
# restrict the area to find the peak in the preferred area
peak_frontal_rh, peak_time = stc_f_omi.in_label(labels_rh_frontal).get_peak()
print(peak_frontal_rh)

# get vertex at the peak
peak_vertex_surf_frontal_rh = stc_f_omi.rh_vertno[peak_frontal_rh]

# get MNI coordinates of the vertex at the peak
coordinate_frontal = mne.vertex_to_mni(stc_f_omi.rh_vertno[peak_vertex_surf_frontal_rh],
                                        hemis=hemi_ind, subject='fsaverage',
                                        subjects_dir=subjects_dir)

print('coordinates frontal: ', coordinate_frontal)

In [ ]:
# plot source estimate with the peak painted blue
kwargs = dict(initial_time=0.1, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all', background='w', 
                      cortex=(211/256,211/256,211/256))
        
brain = stc_f_omi.plot(**kwargs, colormap='Oranges')
brain.add_foci(peak_vertex_surf_frontal_rh, coords_as_verts=True, hemi=hemi, color='blue')
brain.scale_data_colormap(fmin=0, fmid=0.5, fmax=1, transparent=True)

view = 'lateral' #medial
brain.show_view(view);
brain.save_image(sourceFolder +  '100vs80_' + hemi + '_with_maxpoint_'+view+'_frontal.png')


In [ ]:


    
# find the peak on the rh
peak_vertex, peak_time = stc_f_omi.get_peak(hemi=hemi)
print('peak_vertex: ', peak_vertex)

# get the vertex at the peak
peak_vertex_surf = stc_f_omi.rh_vertno[peak_vertex]

# plot the source estimate with the peak painted blue
kwargs = dict(initial_time=0.1, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all', background='w', 
              cortex=(211/256,211/256,211/256)) #, clim=dict(kind='value', pos_lims=(10, 100, 400))
        
brain = stc_f_omi.plot(**kwargs, colormap='Oranges')
brain.scale_data_colormap(fmin=v_mi, fmid=(v_mi+v_ma)/2, fmax=v_ma, transparent=True)

# add the vertex at the peak to the plot
#brain.add_foci(peak_vertex_surf, coords_as_verts=True, hemi=hemi, color='blue')

view = 'lateral' #medial lateral
brain.show_view(view);
brain.save_image(sourceFolder +  'onActivationMaps_omi_8vs10_' + hemi + '_with_maxpoint_'+view+'.png')

# convert vertex to MNI coordinates
coordinate = mne.vertex_to_mni(peak_vertex_surf, hemis=hemi_ind, subject='fsaverage', subjects_dir=subjects_dir)

print('coordinates: ', coordinate)

In [ ]:
#pvalues_o[np.where(pvalues_o <= 0.05)]
#max(fvalues_o_new)

In [ ]:
print(stc_fsave_omi_10_diff.shape)
print(stc_fsave_omi_10_diff[:,0])
print(stc_fsave_omi_8_diff[:,0])
print((stc_fsave_omi_10_diff-stc_fsave_omi_8_diff).shape)

### 3) Compute the difference between 80% and 100%

In [ ]:
import cmasher as cmr
cmap = plt.get_cmap('cmr.seasons')
mask = 300

stc_fsave_omi_diffBetween_10_8 = np.zeros((stc_fsave_omi_10_diff.shape[0], stc_fsave_omi_10_diff.shape[1]),)
for i in range(stc_fsave_omi_10_diff.shape[0]):
    for j in range(stc_fsave_omi_10_diff.shape[1]):
        stc_fsave_omi_diffBetween_10_8[i,j] = stc_fsave_omi_10_diff[i,j]-stc_fsave_omi_8_diff[i,j]

stc_fsave_omi_diffBetween_10_8_avg = np.mean(stc_fsave_omi_diffBetween_10_8, axis=1)
stc_fsave_omi_diffBetween_10_8_avg_masked = stc_fsave_omi_diffBetween_10_8_avg.copy()

for i in range(stc_fsave_omi_diffBetween_10_8_avg_masked.shape[0]):
    if stc_fsave_omi_diffBetween_10_8_avg_masked[i] > 0:
        stc_fsave_omi_diffBetween_10_8_avg_masked[i] = 1
    else:
        stc_fsave_omi_diffBetween_10_8_avg_masked[i] = -1


print('Min of stc_fsave_omi_diffBetween_10_8_avg: ', min(stc_fsave_omi_diffBetween_10_8_avg))
print('Max of stc_fsave_omi_diffBetween_10_8_avg: ', max(stc_fsave_omi_diffBetween_10_8_avg))

stc_fsave_omi_diffBetween_10_8_avg_new = stc_fsave_omi_diffBetween_10_8_avg.copy()
for s in range(stc_fsave_omi_diffBetween_10_8_avg_new.shape[0]):
    if stc_fsave_omi_diffBetween_10_8_avg[s]  < mask and stc_fsave_omi_diffBetween_10_8_avg[s] > -mask:
        stc_fsave_omi_diffBetween_10_8_avg_new[s] = 0
        

print(len(np.where(stc_fsave_omi_diffBetween_10_8_avg_new < 1)[0]))
stc_fsave_omi_diffBetween_10_8_avg_source = prepareInverseSolution_group(stc_fsave_omi_diffBetween_10_8_avg_new,
                                                                         tstep=tstep, tmin_tmp=tminData_cls_tmp)




showResult('fsaverage', sourceFolder, stc_fsave_omi_diffBetween_10_8_avg_source, 
           'omi_diffBetween_10_8_'+inv_sol_method+'_sm='+str(smoothAmount) + 'mask=' + str(mask), 
           minimum=0, mid=500, maximum=1500, transparent=False, cmap='seismic') #, center=0) 
    #seismic twilight_shifted -500, maximum=1000 -  minimum=0, mid=500, maximum=1000

In [ ]:
print(stc_fsave_omi_diffBetween_10_8_avg_new[stc_fsave_omi_diffBetween_10_8_avg_new >= 0].shape)

print(stc_fsave_omi_diffBetween_10_8_avg_new[stc_fsave_omi_diffBetween_10_8_avg_new < 0].shape)

In [ ]:
stc_fsave_omi_10_diff.shape
np.where(stc_fsave_omi_10_diff[:,13] < 0)

In [ ]:
stc_fsave_omi_10_diff.shape[0]

In [ ]:
stc_fsave_omi_diffBetween_10_8_avg[stc_fsave_omi_diffBetween_10_8_avg<-800]

In [ ]:
a = np.mean(stc_fsave_omi_8_diff, axis=1)
a[a > 800].shape